# Lab 03 (n-Puzzle)

In [1]:
from random import choice
import numpy as np
import heapq
from time import time

In [2]:
class Action:
    def __init__(self, pos1: tuple, pos2: tuple) -> None:
        self.pos1 = pos1
        self.pos2 = pos2

In [3]:
def available_actions(state: np.ndarray) -> list[Action]:
    x, y = [int(_[0]) for _ in np.where(state == 0)]
    actions = list()
    if x > 0:
        actions.append(Action((x, y), (x - 1, y)))
    if x < len(state) - 1:
        actions.append(Action((x, y), (x + 1, y)))
    if y > 0:
        actions.append(Action((x, y), (x, y - 1)))
    if y < len(state) - 1:
        actions.append(Action((x, y), (x, y + 1)))
    return actions



def do_action(state: np.ndarray, action: Action) -> np.ndarray:
    new_state = state.copy()
    new_state[action.pos1], new_state[action.pos2] = new_state[action.pos2], new_state[action.pos1]
    return new_state


def create_random_problem(puzzle_dim: int, RANDOMIZE_STEPS=100_000) -> np.ndarray:
    state = np.array([i for i in range(1, puzzle_dim**2)] + [0]).reshape((puzzle_dim, puzzle_dim))
    for _ in range(RANDOMIZE_STEPS):
        state = do_action(state, choice(available_actions(state)))
    return state


def heuristic(state: np.ndarray) -> int:
    """Calculates the total Manhattan distance for a given puzzle state with a penalty for linear conflict."""
    n = len(state)
    manhattan_dist = 0
    linear_conflict_penalty = 0
    
    for (i, j), tile in np.ndenumerate(state):
        if tile == 0:
            continue  # Skip the empty tile
        
        # Compute Manhattan distance        
        goal_i, goal_j = divmod(tile - 1, n)
        manhattan_dist += abs(i - goal_i) + abs(j - goal_j)
        
        # Check for linear conflicts in the same row
        if i == goal_i:  # Tile is in the correct row
            for k in range(j + 1, n):
                other_tile = state[i][k]
                if other_tile != tile and other_tile != 0:
                    other_goal_i = (other_tile - 1) // n
                    if other_goal_i == i and tile > other_tile:  # Both tiles are in their goal row, but out of order
                        linear_conflict_penalty += 2

        # Check for linear conflicts in the same row
        if j == goal_j:  # Tile is in the correct col
            for k in range(i + 1, n):
                other_tile = state[k][j]
                if other_tile != tile and other_tile != 0:
                    other_goal_j = (other_tile - 1) % n
                    if other_goal_j == j and tile > other_tile:  # Both tiles are in their goal col, but out of order
                        linear_conflict_penalty += 2

    return manhattan_dist + linear_conflict_penalty


## Search Strategies

### Node class

In [4]:
class Node:
    def __init__(self, position: np.ndarray, g: int, parent: "Node") -> None:
        self.position = position
        self.g = g  # Cost from start to this node
        self.h = heuristic(position)  # Heuristic estimate to goal
        self.f = self.g + self.h
        self.parent = parent  # Parent node for path reconstruction

    def __hash__(self) -> int:
        return hash(tuple(self.position.reshape(-1)))
    
    def __eq__(self, value: "Node") -> bool:
        return np.array_equal(self.position, value.position)
    
    def __lt__(self, other: "Node") -> bool:
        return (self.f, self.g) < (other.f, other.g)

### A star

In [5]:
def a_star(state: np.ndarray) -> list[np.ndarray]:
    open_set = []
    heapq.heappush(open_set, Node(state, 0, None))
    
    closed_set = set()

    while open_set:
        current_node = heapq.heappop(open_set)
        
        # Goal reached
        if current_node.h == 0:
            path = []
            while current_node:
                path.append(current_node.position)
                current_node = current_node.parent
            return path[::-1]  # Return reversed path from start to goal
        
        closed_set.add(current_node)
        
        # Explore neighbors
        for a in available_actions(current_node.position):
            neighbor_node = Node(do_action(current_node.position, a), current_node.g + 1, current_node)

            # Skip if neighbor is already explored
            if neighbor_node not in closed_set:
                # Add neighbor to priority queue if not visited or found a shorter path
                for node in open_set:
                    if node == neighbor_node:
                        if neighbor_node.g < node.g:
                            open_set.remove(node)
                            heapq.heapify(open_set)
                            heapq.heappush(open_set, neighbor_node)
                        break
                else:
                    heapq.heappush(open_set, neighbor_node)

    return []  # No path found

### Greedy Best First Search

In [6]:
def greedy_best_first(state: np.ndarray) -> list[np.ndarray]:
    visited = set()
    stack = []
    heapq.heappush(stack, Node(state, 0, None))

    while stack:
        current = heapq.heappop(stack)
        visited.add(current)

        if current.h == 0:
            path = []
            while current:
                path.append(current.position)
                current = current.parent
            return path[::-1]  # Return reversed path from start to goal
        
        for a in available_actions(current.position):
            neighbor = Node(do_action(current.position, a), 0, current)
            if neighbor not in visited:
                heapq.heappush(stack, neighbor)

    return None  # No solution found

## Solve

### Results

In [8]:
prob = np.array([7,8,3,4,6,0,1,2,5]).reshape((3, 3))
sol = a_star(prob)
end = time()

KeyboardInterrupt: 

In [7]:
for PUZZLE_DIM in range(2, 8):
    print(f"Dimensions: {PUZZLE_DIM}x{PUZZLE_DIM} -> ", end="")
    prob = create_random_problem(PUZZLE_DIM)
    start = time()
    sol = greedy_best_first(prob)
    end = time()
    print(f"Steps: {len(sol) - 1} ({(end - start):.3f}s)", end="")
    if PUZZLE_DIM < 4:
        sol = a_star(prob)
        print(f" (Optimal #steps: {len(sol) - 1})", end="")
    print()

Dimensions: 2x2 -> Steps: 0 (0.000s) (Optimal #steps: 0)
Dimensions: 3x3 -> Steps: 32 (0.009s) (Optimal #steps: 18)
Dimensions: 4x4 -> Steps: 70 (0.051s)
Dimensions: 5x5 -> Steps: 274 (0.414s)
Dimensions: 6x6 -> Steps: 642 (7.392s)
Dimensions: 7x7 -> 

KeyboardInterrupt: 

### Test yourself!

In [ ]:
PUZZLE_DIM = 3
prob = create_random_problem(PUZZLE_DIM)
print(f"Problem:\n{prob}")

if PUZZLE_DIM < 2:
    print("Empty problem!")
elif PUZZLE_DIM < 4:
    print("Optimal solution!")
    sol = a_star(prob)
    print(f"Steps: {len(sol) - 1}\nPath:")
    print(np.array(sol))
else:
    print(f"Approximated solution{' (this mat take a while...)' if PUZZLE_DIM > 7 else ''}!")
    sol = greedy_best_first(prob)
    print(f"Steps: {len(sol) - 1}\nPath:")
    print(np.array(sol))


Problem:
[[8 5 3]
 [4 0 1]
 [6 2 7]]
Optimal solution!
Steps: 24
Path:
[[[8 5 3]
  [4 0 1]
  [6 2 7]]

 [[8 5 3]
  [4 2 1]
  [6 0 7]]

 [[8 5 3]
  [4 2 1]
  [0 6 7]]

 [[8 5 3]
  [0 2 1]
  [4 6 7]]

 [[0 5 3]
  [8 2 1]
  [4 6 7]]

 [[5 0 3]
  [8 2 1]
  [4 6 7]]

 [[5 2 3]
  [8 0 1]
  [4 6 7]]

 [[5 2 3]
  [8 1 0]
  [4 6 7]]

 [[5 2 0]
  [8 1 3]
  [4 6 7]]

 [[5 0 2]
  [8 1 3]
  [4 6 7]]

 [[5 1 2]
  [8 0 3]
  [4 6 7]]

 [[5 1 2]
  [0 8 3]
  [4 6 7]]

 [[0 1 2]
  [5 8 3]
  [4 6 7]]

 [[1 0 2]
  [5 8 3]
  [4 6 7]]

 [[1 2 0]
  [5 8 3]
  [4 6 7]]

 [[1 2 3]
  [5 8 0]
  [4 6 7]]

 [[1 2 3]
  [5 0 8]
  [4 6 7]]

 [[1 2 3]
  [5 6 8]
  [4 0 7]]

 [[1 2 3]
  [5 6 8]
  [4 7 0]]

 [[1 2 3]
  [5 6 0]
  [4 7 8]]

 [[1 2 3]
  [5 0 6]
  [4 7 8]]

 [[1 2 3]
  [0 5 6]
  [4 7 8]]

 [[1 2 3]
  [4 5 6]
  [0 7 8]]

 [[1 2 3]
  [4 5 6]
  [7 0 8]]

 [[1 2 3]
  [4 5 6]
  [7 8 0]]]
